In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
#from sklearn import cluster
from kneed import KneeLocator

In [2]:
#read the pkl file for movies
df = pd.read_pickle('./data/full_movies.pkl')
#read the pkl file for shows
df_2 = pd.read_pickle('./ABC/full_shows.pkl')

In [3]:
df

,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title
11,11,Family,Ballet Now,NaN,Ballet Now,This film captures the creative process behind...,This film captures the creative process behind...,"[ABC TV Plus, DOCUMENTARY, ARTS & CULTURE, FAM...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW269...,[],"[abc2, docs, factual, arts, ballet, dance, cul...",2021-06-30 21:59:27,M,3555.0,NaN,,Ballet Now
35,35,Family,Zog,NaN,Zog,"Based on the picture book by Julia Donaldson, ...","Based on the picture book by Julia Donaldson, ...","[ABC Kids, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW124...,"[Director Max Lang, Daniel Snaddon]","[abc4kids, 4feature, 4featured, abcme-shoulder...",2020-04-17 11:45:30,G,1621.0,NaN,,Zog
111,111,Family,Armadillo: Narrated By David Attenborough,NaN,Armadillo: Narrated By David Attenborough,In the heart of Brazil lives an animal so elus...,In the heart of Brazil lives an animal so elus...,"[ABC TV, DOCUMENTARY, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW311...,[Host Sir David Attenborough],"[abc1, uk, docs, factual, nature, wildlife, en...",2021-09-19 19:41:30,PG,3134.0,NaN,,Armadillo: Narrated By David Attenborough
137,137,Family,Stick Man,NaN,Stick Man,Stick Man lives in the family tree with his St...,Stick Man lives in the family tree with his St...,"[ABC Kids, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW002...,"[Director Jeroen Jaspaert, Daniel Snaddon]","[abc4kids, family-viewing, 4xmas-features, abc...",2020-04-17 11:44:30,G,1626.0,NaN,,Stick Man
158,158,Family,The Snail And The Whale,NaN,The Snail And The Whale,"A beautiful tale of an unlikely friendship, a ...","A beautiful tale of an unlikely friendship, a ...","[ABC Kids, ABC ME, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW177...,"[Director Max Lang, Daniel Snaddon]","[abc4kids, 4featured, 4feature, 4family, 4uppe...",2020-04-17 11:46:30,G,1601.0,NaN,,The Snail And The Whale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2255,2255,Education,What's For Dinner? Lunar New Year (Simplified ...,NaN,What's For Dinner? Lunar New Year (Simplified ...,Welcome to Penny's place for Lunar New Year! I...,Welcome to Penny's place for Lunar New Year! I...,"[ABC ME, ABC TV, EDUCATION, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/iv/IV210...,[],"[abc3, education, iview, abc1, family-viewing]",2021-02-05 06:00:00,G,783.0,NaN,,What's For Dinner? Lunar New Year (Simplified ...
2305,2305,Education,BTN: Antarctica Special,NaN,BTN: Antarctica Special,A BTN Special all about Antarctica; from the p...,A BTN Special all about Antarctica from the pe...,"[ABC ME, EDUCATION, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/ne/NE190...,[],"[abc3, education, primary-humanities, secondar...",2019-09-24 10:00:00,NaN,1228.0,NaN,,BTN: Antarctica Special
2354,2354,Education,Life In Medieval Europe,NaN,Life In Medieval Europe,"Set in 1350, this medieval drama unfolds throu...","Set in 1350, this medieval drama unfolds throu...","[ABC ME, EDUCATION]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW027...,[],"[abc3, education, secondary-history]",2020-03-16 09:00:00,G,1475.0,NaN,,Life In Medieval Europe
2359,2359,Education,Montserrat: Living With Volcanoes,NaN,Montserrat: Living With Volcanoes,This program deals with the impacts of the vol...,This program deals with the impacts of the vol...,"[ABC ME, EDUCATION]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW113...,[],"[abc3, education, secondary-geography]",2017-08-17 11:00:00,G,2750.0,NaN,,Montserrat: Living With Volcanoes


In [4]:
df_2

,id,category,description,description2,directors_actors,publication_date,rating,duration_sec,episodes_ids,tags,tags2
title,,,,,,,,,,,
199 Little Heroes,11529,Education,The journey to school is a very special type o...,"[Enjo lives in Quinten, a forest glade in the ...",[],2021-06-01 06:20:00,G,"[311.0, 311.0, 312.0, 311.0, 311.0, 320.0, 311...","[12715, 12925, 12930, 13052, 13187, 13409, 134...","[ABC ME, EDUCATION]","[environment-day, abc3, primary-humanities, re..."
7.30,11530,News,Leigh Sales presents Australia's leading night...,[The devastation caused by the recent floods i...,[],2022-03-14 20:00:00,None,"[1906.0, 1894.0, 1879.0, 1935.0, 1891.0, 1822....","[657, 663, 756, 809, 860, 981, 1050, 1067, 114...","[ABC TV, ABC NEWS]","[pandemic, analysis, current-affairs, news24, ..."
7.30 Mark Humphries Satire,11531,News,Satirist Mark Humphries brings his unique pers...,[Satirist Mark Humphries goes inside the Healt...,[],2021-05-11 14:00:00,None,"[145.0, 110.0, 118.0, 136.0, 202.0, 115.0, 241...","[634, 643, 853, 883, 1001, 1082, 1085, 1275, 1...",[ABC NEWS],"[humphries, news24, duty, of, satire, sussan, ..."
7.30: The Interviews,11532,News,Revisit some of 7.30's most compelling convers...,[Musician Dave Grohl sits down with 7.30's Lei...,[],2022-03-08 20:00:00,None,[1030.0],[1009],[ABC NEWS],"[news, news24]"
72 Cutest Animals,11533,Family,The animal kingdom continually astounds us wit...,[Cute is a notion that can mean many things to...,[],2022-03-05 15:03:06,G,"[1616.0, 1616.0, 1620.0, 1620.0, 1614.0, 1620....","[2043, 2160, 2751, 2833, 2997, 3118, 3213]","[ABC ME, FAMILY]","[family-viewing, family-viewing-abcme, abc3]"
...,...,...,...,...,...,...,...,...,...,...,...
hoopla doopla!,12201,Kids,hoopla doopla! is a live action preschool show...,[Jango dresses up as a king and his friends mi...,[],2022-01-28 06:15:10,G,"[720.0, 720.0, 720.0, 720.0, 720.0, 720.0, 720...","[4015, 4097, 4184, 4189, 4436, 4443, 4821, 483...",[ABC Kids],"[ps-birthday, 4sing-dance, 4play-time, 4xmas, ..."
hoopla doopla! (Mandarin),12202,Kids,hoopla doopla! is a live action preschool show...,[Jango is determined to be picked to be in the...,[],2022-01-28 06:35:20,G,"[720.0, 720.0, 720.0, 720.0, 720.0, 720.0, 720...","[3902, 4111, 4124, 4140, 4252, 4263, 4432, 445...",[ABC Kids],"[abc4kids, education, 4liveaction, forever]"
storyTree,12203,Kids,"Join storyteller, Miss Treeleaf, in an imagina...","[When little red hen bakes a cake, her lazy fr...",[],2020-03-20 14:11:00,G,"[210.0, 210.0, 210.0, 210.0, 210.0, 210.0, 210...","[3968, 4046, 4053, 4266, 4274, 4363, 4568, 472...","[ABC Kids, EDUCATION]","[4halloween, 4story, abc4kids, education, prim..."


In [5]:
#read json file activity for movies
activity = pd.read_json('./activities.json')

In [6]:
activity.head()
activity.groupby('user_id').count()

,content_id,activity,datetime
user_id,,,
1,525,525,525
2,508,508,508
3,504,504,504
4,475,475,475
5,488,488,488


## TF-IDF + kmeans clustering

In [7]:
#tf-idf and kmeans for all movies
# Convert the text data to a matrix of TF-IDF features
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(df['description'])


inertia = []
#calculate the optimal number of clusters
K = range(1, 20)
for k in K:
    kmeanModel = KMeans(n_clusters=k,random_state=0)
    kmeanModel.fit(tfidf)
    inertia.append(kmeanModel.inertia_)


kn = KneeLocator(K, inertia, curve='convex', direction='decreasing')
print(kn.knee)


13


In [8]:
#tf-idf and kmeans for all tv-shows
# Convert the text data to a matrix of TF-IDF features
tfidf_vectorizer_shows = TfidfVectorizer(stop_words='english')
tfidf_shows = tfidf_vectorizer_shows.fit_transform(df_2['description'])


inertia = []
#calculate the optimal number of clusters
K = range(1, 20)
for k in K:
    kmeanModel = KMeans(n_clusters=k,random_state=0)
    kmeanModel.fit(tfidf_shows)
    inertia.append(kmeanModel.inertia_)


kn = KneeLocator(K, inertia, curve='convex', direction='decreasing')
print(kn.knee)

9


In [9]:
# Cluster the documents using KMeans
num_clusters = 13
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf)

# Assign cluster labels to each document
clusters = km.labels_.tolist()
df['cluster'] = clusters

In [10]:
# Cluster the documents using KMeans
num_clusters_shows = 9
km_shows = KMeans(n_clusters=num_clusters_shows)
km_shows.fit(tfidf_shows)

# Assign cluster labels to each document
clusters_shows = km_shows.labels_.tolist()
df_2['cluster'] = clusters_shows
df_2.reset_index(inplace=True)
df_2.head()

,title,id,category,description,description2,directors_actors,publication_date,rating,duration_sec,episodes_ids,tags,tags2,cluster
0,199 Little Heroes,11529,Education,The journey to school is a very special type o...,"[Enjo lives in Quinten, a forest glade in the ...",[],2021-06-01 06:20:00,G,"[311.0, 311.0, 312.0, 311.0, 311.0, 320.0, 311...","[12715, 12925, 12930, 13052, 13187, 13409, 134...","[ABC ME, EDUCATION]","[environment-day, abc3, primary-humanities, re...",0
1,7.30,11530,News,Leigh Sales presents Australia's leading night...,[The devastation caused by the recent floods i...,[],2022-03-14 20:00:00,None,"[1906.0, 1894.0, 1879.0, 1935.0, 1891.0, 1822....","[657, 663, 756, 809, 860, 981, 1050, 1067, 114...","[ABC TV, ABC NEWS]","[pandemic, analysis, current-affairs, news24, ...",6
2,7.30 Mark Humphries Satire,11531,News,Satirist Mark Humphries brings his unique pers...,[Satirist Mark Humphries goes inside the Healt...,[],2021-05-11 14:00:00,None,"[145.0, 110.0, 118.0, 136.0, 202.0, 115.0, 241...","[634, 643, 853, 883, 1001, 1082, 1085, 1275, 1...",[ABC NEWS],"[humphries, news24, duty, of, satire, sussan, ...",2
3,7.30: The Interviews,11532,News,Revisit some of 7.30's most compelling convers...,[Musician Dave Grohl sits down with 7.30's Lei...,[],2022-03-08 20:00:00,None,[1030.0],[1009],[ABC NEWS],"[news, news24]",5
4,72 Cutest Animals,11533,Family,The animal kingdom continually astounds us wit...,[Cute is a notion that can mean many things to...,[],2022-03-05 15:03:06,G,"[1616.0, 1616.0, 1620.0, 1620.0, 1614.0, 1620....","[2043, 2160, 2751, 2833, 2997, 3118, 3213]","[ABC ME, FAMILY]","[family-viewing, family-viewing-abcme, abc3]",8


In [11]:
australian = 'Australia|australia|australian|Australian|Aboriginal|Indigenous|First People'

In [12]:
expat_library = df[df['tags2'].str.contains(australian) | df['description'].str.contains(australian) | df['description2'].str.contains(australian) | df['title'].str.contains(australian)
]
expat_library.head()

,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title,cluster
237,237,Family,Romeo & Juliet,NaN,Romeo & Juliet,"Graeme Murphy's Romeo and Juliet, performed by...","Graeme Murphy's Romeo and Juliet, performed by...","[ABC TV Plus, ABC ARTS, ARTS & CULTURE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW230...,[],"[abc2, arts, abcarts, ballet, entertainment, c...",2021-06-18 07:00:00,PG,7168.0,NaN,,Romeo & Juliet,4
404,404,Family,Can We Save The Reef?,NaN,Can We Save The Reef?,An epic story of Australian and international ...,An epic story of Australian and international ...,"[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/sc/SC160...,[],"[abc1, docs, nature, science, science-week, cl...",2018-05-12 14:58:55,G,3428.0,NaN,,Can We Save The Reef?,0
424,424,Family,Stackorama!,NaN,Stackorama!,"Jaydyn Coggins, the best cup stacker in Austra...","Jaydyn Coggins, the best cup stacker in Austra...","[ABC TV Plus, DOCUMENTARY, SPORT, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/do/DO191...,[],"[abc2, docs, factual, sport, endearing, disabi...",2021-03-31 21:45:07,PG,3586.0,NaN,,Stackorama!,2
484,484,Family,Australia: The Wild Top End,NaN,Australia: The Wild Top End,Come with us on an epic journey through some o...,Come with us on an epic journey through some o...,"[ABC ME, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW284...,[],"[abc3, 3featured, family-viewing-abcme, family...",2022-03-16 09:45:00,PG,2602.0,NaN,,Australia: The Wild Top End,10
595,595,Family,Freeman,NaN,Freeman,The story of a nation coming together around I...,The story of a nation coming together around I...,"[ABC TV, ABC ME, INDIGENOUS, DOCUMENTARY, SPOR...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF190...,[Director Laurence Billiet],"[abc1, australia, indigenous, docs, factual, f...",2021-06-30 07:05:00,G,3480.0,NaN,,Freeman,7


In [13]:
expat_library_shows = df_2[df_2['tags2'].str.contains(australian) | df_2['description'].str.contains(australian) | df_2['title'].str.contains(australian)
]
expat_library_shows.shape

(23, 13)

In [14]:
#reset index for df_2
df_2.reset_index(inplace=True)
#drop the column 'index'
df_2.drop(columns=['index'], inplace=True)
df_2.head()

,title,id,category,description,description2,directors_actors,publication_date,rating,duration_sec,episodes_ids,tags,tags2,cluster
0,199 Little Heroes,11529,Education,The journey to school is a very special type o...,"[Enjo lives in Quinten, a forest glade in the ...",[],2021-06-01 06:20:00,G,"[311.0, 311.0, 312.0, 311.0, 311.0, 320.0, 311...","[12715, 12925, 12930, 13052, 13187, 13409, 134...","[ABC ME, EDUCATION]","[environment-day, abc3, primary-humanities, re...",0
1,7.30,11530,News,Leigh Sales presents Australia's leading night...,[The devastation caused by the recent floods i...,[],2022-03-14 20:00:00,None,"[1906.0, 1894.0, 1879.0, 1935.0, 1891.0, 1822....","[657, 663, 756, 809, 860, 981, 1050, 1067, 114...","[ABC TV, ABC NEWS]","[pandemic, analysis, current-affairs, news24, ...",6
2,7.30 Mark Humphries Satire,11531,News,Satirist Mark Humphries brings his unique pers...,[Satirist Mark Humphries goes inside the Healt...,[],2021-05-11 14:00:00,None,"[145.0, 110.0, 118.0, 136.0, 202.0, 115.0, 241...","[634, 643, 853, 883, 1001, 1082, 1085, 1275, 1...",[ABC NEWS],"[humphries, news24, duty, of, satire, sussan, ...",2
3,7.30: The Interviews,11532,News,Revisit some of 7.30's most compelling convers...,[Musician Dave Grohl sits down with 7.30's Lei...,[],2022-03-08 20:00:00,None,[1030.0],[1009],[ABC NEWS],"[news, news24]",5
4,72 Cutest Animals,11533,Family,The animal kingdom continually astounds us wit...,[Cute is a notion that can mean many things to...,[],2022-03-05 15:03:06,G,"[1616.0, 1616.0, 1620.0, 1620.0, 1614.0, 1620....","[2043, 2160, 2751, 2833, 2997, 3118, 3213]","[ABC ME, FAMILY]","[family-viewing, family-viewing-abcme, abc3]",8


In [15]:
expat_library_shows= df_2[df_2['tags2'].str.contains(australian) | df_2['description'].str.contains(australian) | df_2['description2'].str.contains(australian) | df_2['title'].str.contains(australian)
]
expat_library_shows.head()

,title,id,category,description,description2,directors_actors,publication_date,rating,duration_sec,episodes_ids,tags,tags2,cluster
56,Australia Debates,11585,Panel Discussion,The funniest minds in Australia debate the fin...,"[Tom Cashman, Demi Lardner and Nat Damena argu...",[Host Nikki Britton],2021-06-28 21:33:30,M,"[2579.0, 2605.0, 2492.0]","[26, 49, 225]","[ABC TV Plus, PANEL & DISCUSSION]","[a:established-parents, abc2, ns:be-informed, ...",4
57,Australia Remastered: Australia's Oceans,11586,Family,Explore the incredible diversity of ocean wild...,[The Southern Ocean is the only ocean that str...,[Host Aaron Pedersen],2020-10-25 19:03:00,G,"[3230.0, 3195.0, 3185.0]","[1933, 2047, 2104]","[ABC TV, ABC ME, DOCUMENTARY, SCIENCE, FAMILY]","[a:retirees, docs, wildlife, ns:be-informed, i...",3
58,Australia Remastered: Forces of Nature,11587,Family,"Hosted and narrated by Aaron Pedersen, this se...",[Cyclones burst upon Australia's tropical coas...,[Host Aaron Pedersen],2022-01-30 18:04:10,G,"[3230.0, 3260.0, 3195.0, 3240.0]","[1861, 2547, 2688, 3738]","[ABC TV, DOCUMENTARY, REGIONAL AUSTRALIA, FAMILY]","[docs, environment, regional, natural-disaster...",4
59,Australia Remastered: Nature's Great Divide,11588,Family,The hard line that separates the natural world...,[The narrow strait between the Indonesian isla...,[Host Aaron Pedersen],2021-12-07 21:27:05,G,"[3240.0, 3240.0, 3200.0]","[2840, 3476, 3601]","[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]","[ocean, docs, environment, family-viewing, wil...",3
60,Australia Remastered: Wild Australians,11589,Family,Explore some of Australia's most iconic and fa...,"[Echidnas and platypuses are unique, the only ...",[Host Aaron Pedersen],2020-08-30 18:57:00,G,"[3310.0, 3330.0, 3285.0, 3185.0, 3230.0, 3294.0]","[1990, 2401, 2805, 2863, 3397, 3614]","[ABC TV, ABC ME, DOCUMENTARY, FAMILY]","[a:retirees, docs, wildlife, science-week, ns:...",2


In [16]:
# Print the top 10 terms for each cluster
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = tfidf_vectorizer.get_feature_names()
for i in range(num_clusters):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :13]:
        print(' %s' % terms[ind], end='')
    print()



Cluster 0: scenes making look attenborough documentary david natural history series film process wonder stage
Cluster 1: gospel stunning music mark win monastery manchester lisser semi gorton talented final celebration
Cluster 2: new people disability world celebrate christmas films day school desert story metal jaydyn
Cluster 3: attenborough david sir takes reveals magical ancient night journey join 2009 young london
Cluster 4: performed ballet australian music orchestra opera australia victoria love symphony costumes story production
Cluster 5: artist family political personal stick human man challenge intimate freedom stop artistic sarah
Cluster 6: home identity young story fielding culture fashion church tell paquita town come dance
Cluster 7: life love story year old age girl beach friends time summer great set
Cluster 8: seafood freshest welcome dumplings homemade markets salad colourful tossing tradition dragons starts fish
Cluster 9: special live work research episode people fi

/Users/ekaterinamazur/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [17]:
# Print the top 10 terms for each cluster
order_centroids_shows = km_shows.cluster_centers_.argsort()[:, ::-1]
terms_shows = tfidf_vectorizer_shows.get_feature_names()
for i in range(num_clusters_shows):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids_shows[i, :13]:
        print(' %s' % terms_shows[ind], end='')
    print()

Cluster 0: stages maths secrets english make science new life short stage join questions french
Cluster 1: family old year life adventures action epic new friends search embark world face
Cluster 2: series world stage science explore episode young english animated featuring tell iconic features
Cluster 3: life people series revealing stories favourite changing new abc takes stages following history
Cluster 4: australia art explores world series australian national identity artists history people survive unique
Cluster 5: adventures friends time children story dr fun songs live team new little books
Cluster 6: news abc stories issues reporting brings today original investigations exclusive presented matter latest
Cluster 7: australian teachers lessons minute teach skills fundamental science aboriginal water land year august
Cluster 8: big animals join kids ted world learn play adventures school ready animal best


/Users/ekaterinamazur/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [18]:
joined_activity = pd.merge(df, activity, left_on='id', right_on='content_id', how='right')
#make from list in tag2 column to string and delete [] in the string
joined_activity['tags2'] = joined_activity['tags2'].apply(lambda x: str(x).replace('[', '').replace(']', ''))
#drop NaN values in id column
joined_activity.dropna(subset=['id'], inplace=True)
#drop columns that are not needed
joined_activity.drop(['series', 'episode_name', 'description2', 'publication_date', 'rating', 'season', 'episode', 'episode_title', 'datetime', 'duration_sec', 'tags'], axis=1, inplace=True)
joined_activity.head()

,id,category,title,description,image,directors_actors,tags2,cluster,content_id,activity,user_id
7,638.0,Documentary,Bad Influencer,"By 23, Belle Gibson had cultivated a global fo...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW292...,[Cast Belle Gibson],"'abc1', 'australia', 'docs', 'factual', 'healt...",0.0,638,Dislike,2
13,2030.0,Family,The Gruffalo,Tells the magical tale of a mouse who takes a ...,https://cdn.iview.abc.net.au/thumbs/i/zx/ZX058...,"[Director Max Lang, Jakob Schuh]","'4family', '4upper', '4feature', 'abc4kids', '...",3.0,2030,Play,4
23,1759.0,Family,The Making Of The Penguin King With David Atte...,A behind-the-scenes look at the documentary wh...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW259...,[Host Sir David Attenborough],"'abc2', 'uk', 'docs', 'factual', 'animals', 'n...",0.0,1759,Play,4
31,681.0,Arts,Opera On Sydney Harbour: La Boheme,Experience the romance of the original bohemia...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW230...,[Director Andy Morton],"'abc2', 'abcarts', 'arts', 'opera', 'performan...",4.0,681,Like,2
35,746.0,Documentary,Step Into Paradise,The extraordinary story of iconic fashion desi...,https://cdn.iview.abc.net.au/thumbs/i/ac/AC190...,"[Cast Jenny Kee, Linda Jackson]","'abc1', 'docs', 'factual', 'abcarts', 'arts', ...",6.0,746,Dislike,2


In [19]:
joined_activity_shows = pd.merge(df_2, activity, left_on='id', right_on='content_id', how='right')
#make from list in tag2 column to string and delete [] in the string
joined_activity_shows['tags2'] = joined_activity_shows['tags2'].apply(lambda x: str(x).replace('[', '').replace(']', ''))
#drop NaN values in id column
joined_activity_shows.dropna(subset=['id'], inplace=True)
#drop columns that are not needed
joined_activity_shows.drop([ 'description2', 'publication_date', 'rating', 'datetime', 'duration_sec', 'tags'], axis=1, inplace=True)
joined_activity_shows.head()

,title,id,category,description,directors_actors,episodes_ids,tags2,cluster,content_id,activity,user_id
1000,Live At The Apollo,11844.0,Comedy,Live At The Apollo is back for a 16th series w...,[],"[8191, 8199, 8200, 8205, 8221, 8234, 8281, 830...","'abc2', 'uk', 'a:young-sink-dink', 'live-comed...",2.0,11844,Like,3
1003,This Place: Artist Series,12136.0,Arts,Six Aboriginal and Torres Strait Islander arti...,[],"[11908, 12051, 12369, 12468, 12620]","'docs', 'regional', 'short', 'education', 'ns:...",4.0,12136,Like,1
1010,Noddy Toyland Detective,11915.0,Kids,"Together with his best buds Revs and Bumpy, No...",[],"[3852, 3972, 4038, 4209, 4212, 4487, 4502, 468...","'abc4kids', '4adventure', '4lower', '4featured'",5.0,11915,Like,2
1014,Justine Clarke: Songs To Make You Smile,11807.0,Kids,Playschool presenter Justine Clarke sings song...,[],"[4246, 4471, 6168, 6985, 7849]","'4song', '4shorts', '4liveaction', '4girls', '...",5.0,11807,Like,2
1015,The Stickie Gang,12111.0,Kids,"Angie, Eddie and Mia are best friends who live...",[],"[4172, 4439, 5479, 5815, 6223, 7068, 7493, 7760]","'abc4kids', 'abc3', 'primary-maths', 'education'",5.0,12111,Play,1


In [20]:
#select rows where activity is 'Like'
expat_data_likes = joined_activity[joined_activity['activity'] == 'Like']
expat_data_likes.sort_values(by=['user_id'])

,id,category,title,description,image,directors_actors,tags2,cluster,content_id,activity,user_id
991,1015.0,Drama,The Way We Were,A love story that begins with the attraction o...,https://cdn.iview.abc.net.au/thumbs/i/zy/ZY933...,[Director Sydney Pollack],"'abc1', 'abc2', 'drama', 'classic', 'film', 'f...",7.0,1015,Like,1
293,136.0,Drama,Visible,"A teenage girl who feels left out and ignored,...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF210...,[Director Genevieve Clay-Smith],"'abc1', 'abc2', 'drama', 'diversity', 'cald', ...",7.0,136,Like,1
289,505.0,Drama,Tommy,"A psychosomatically deaf, dumb and blind boy d...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW301...,[Director Ken Russell],"'abc1', 'abc2', 'drama', 'fantasy', 'satire', ...",0.0,505,Like,1
348,2722.0,Kids,The Highway Rat,An animated swash-buckler based on the much-lo...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW081...,[Director Jeroen Jaspaert],"'abc4kids', '4featured', '4feature', '4family'...",12.0,2722,Like,1
568,137.0,Family,Stick Man,Stick Man lives in the family tree with his St...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW002...,"[Director Jeroen Jaspaert, Daniel Snaddon]","'abc4kids', 'family-viewing', '4xmas-features'...",5.0,137,Like,1
607,1275.0,Family,Stargazing: Moon and Beyond,Stargazing celebrates the 50th anniversary of ...,https://cdn.iview.abc.net.au/thumbs/i/sc/SC180...,"[Host Julia Zemiro, Brian Cox]","'abc1', 'science', 'docs', 'space', 'family-vi...",10.0,1275,Like,1
384,942.0,Drama,Little Fish,"After a turbulent time in her 20s, Tracy Heart...",https://cdn.iview.abc.net.au/thumbs/i/zy/ZY970...,[Director Rowan Woods],"'abc1', 'abc2', 'drama', 'film', 'feature-leng...",11.0,942,Like,1
101,237.0,Family,Romeo & Juliet,"Graeme Murphy's Romeo and Juliet, performed by...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW230...,[],"'abc2', 'arts', 'abcarts', 'ballet', 'entertai...",4.0,237,Like,1
637,834.0,Drama,La Bamba,"Ritchie Valens, a 17-year-old Mexican migrant,...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW300...,[Director Luis Valdez],"'abc1', 'abc2', 'drama', 'biopic', 'film', 'fe...",7.0,834,Like,1
671,1451.0,Family,Superworm,Animated adaptation of the children's book by ...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW299...,[Narrator Olivia Colman],"'abc4kids', '4featured', '4feature', '4family'...",12.0,1451,Like,2


In [21]:
#select rows where activity is 'Like'
expat_data_shows_likes = joined_activity_shows[joined_activity_shows['activity'] == 'Like']
expat_data_shows_likes.sort_values(by=['user_id'])

,title,id,category,description,directors_actors,episodes_ids,tags2,cluster,content_id,activity,user_id
1595,Further Back In Time For Dinner,11737.0,Family,An ordinary family travels back in time 120 ye...,[Host Annabel Crabb],"[1802, 2483, 2586, 3424, 3777]","'a:retirees', 'docueseries', 'docs', 'period',...",1.0,11737,Like,1
2286,ABC Late News,11537.0,News,End your day informed and up-to-date with the ...,[],"[709, 798, 1203, 1498]","'news', 'news24', 'bulletin'",6.0,11537,Like,1
1208,Belgravia,11615.0,Drama,When the Trenchards accept an invitation to a ...,"[Richard Goulding, Harriet Walter, Cast Tamsin...","[10575, 10772, 11033, 11284, 11356, 11357]","'romance', 'a:retirees', 'binge-worthy', 'dram...",0.0,11615,Like,1
2256,Australia Remastered: Wild Australians,11589.0,Family,Explore some of Australia's most iconic and fa...,[Host Aaron Pedersen],"[1990, 2401, 2805, 2863, 3397, 3614]","'a:retirees', 'docs', 'wildlife', 'science-wee...",2.0,11589,Like,1
1252,7.30 Mark Humphries Satire,11531.0,News,Satirist Mark Humphries brings his unique pers...,[],"[634, 643, 853, 883, 1001, 1082, 1085, 1275, 1...","'humphries', 'news24', 'duty', 'of', 'satire',...",2.0,11531,Like,1
...,...,...,...,...,...,...,...,...,...,...,...
1957,The Silence (UK),12109.0,Drama,"An eighteen-year-old, struggling to integrate ...","[Cast Genevieve Barr, Gina McKee, Dervla Kirwa...","[10581, 10939, 11027, 11706]","'a:established-parents', 'a:worki', 'a:retiree...",1.0,12109,Like,5
1475,Tik Tak,12141.0,Kids,Tik Tak stimulates children's cognitive develo...,[],"[3857, 3861, 3921, 3926, 3990, 4006, 4115, 411...","'abc4kids', '4puggle'",8.0,12141,Like,5
1877,Mini Lessons: Science Primary,11879.0,Education,"In these ten-minute lessons, Australian teache...",[],"[12783, 14366]","'science', 'abc3', 'primary-maths', 'education'",7.0,11879,Like,5
2007,Odd Squad,11920.0,Family,The Odd Squad Mobile Unit is back for more glo...,"[Jayce Alexander, Gavin MacIver-Wright, Cast A...","[1798, 1806, 1825, 1831, 1847, 1854, 1859, 186...","'investigation', 'mystery', 'adventure', 'fami...",1.0,11920,Like,5


In [22]:
#top-3 clusters for each user in expat_data_likes
top3 = expat_data_likes.groupby('user_id')['cluster'].value_counts().groupby(level=0).nlargest(3).reset_index(level=1, drop=True)
#make from series to dataframe
top3 = top3.to_frame()
#drop the 3rd column
top3 = top3.drop(columns=['cluster'])
#make user_id column as index
top3 = top3.reset_index()
top3

,user_id,cluster
0,1,7.0
1,1,0.0
2,1,4.0
3,2,12.0
4,2,2.0
5,2,4.0
6,3,7.0
7,3,11.0
8,3,0.0
9,4,5.0


In [23]:
expat_data_shows_likes.sort_values(by=['user_id'])
#top-3 clusters for each user in expat_data_likes
top3_show = expat_data_shows_likes.groupby('user_id')['cluster'].value_counts().groupby(level=0).nlargest(3).reset_index(level=1,
                                                                                                              drop=True)
#make from series to dataframe
top3_show = top3_show.to_frame()
#drop the 3rd column
top3_show = top3_show.drop(columns=['cluster'])
#make user_id column as index
top3_show = top3_show.reset_index()
top3_show

,user_id,cluster
0,1,5.0
1,1,2.0
2,1,4.0
3,2,5.0
4,2,1.0
5,2,4.0
6,3,4.0
7,3,2.0
8,3,1.0
9,4,5.0


In [24]:
#filter expat_data_likes dataframe by another dataframe with two columns: user_id and cluster
#this is only activity 'Like' for top-3 clusters for each user
# expat_data_top = pd.merge(expat_data_likes, top3, on=['user_id', 'cluster'])
# expat_data_top.sort_values(by=['user_id'])

In [25]:
#join top 3 clusters for each user with expat_library
recommended= pd.merge(top3, expat_library, on=['cluster'], how='left')
recommended

,user_id,cluster,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title
0,1,7.0,595,Family,Freeman,NaN,Freeman,The story of a nation coming together around I...,The story of a nation coming together around I...,"[ABC TV, ABC ME, INDIGENOUS, DOCUMENTARY, SPOR...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF190...,[Director Laurence Billiet],"[abc1, australia, indigenous, docs, factual, f...",2021-06-30 07:05:00,G,3480.0,NaN,,Freeman
1,1,7.0,1979,Family,Sea Lions: Life By A Whisker,NaN,Sea Lions: Life By A Whisker,Take an intimate journey inside the colony of ...,Take an intimate journey inside the colony of ...,"[ABC ME, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW284...,[Narrator Sam Neill],"[abc3, family-viewing-abcme, 3featured, family...",2022-03-16 09:45:00,G,2439.0,NaN,,Sea Lions: Life By A Whisker
2,1,7.0,32,Documentary,Fly Me To The Moon,NaN,Fly Me To The Moon,The Moon Landing was the iconic moment of our ...,The Moon Landing was the iconic moment of our ...,"[ABC NEWS, DOCUMENTARY, SCIENCE]",https://cdn.iview.abc.net.au/thumbs/i/nc/NC193...,[],"[news24, news, space, docs, science, featured]",2021-04-07 13:00:00,PG,3480.0,NaN,,Fly Me To The Moon
3,1,7.0,71,Documentary,Ready Together,NaN,Ready Together,Craig Reucassel presents the stories of two co...,Craig Reucassel presents the stories of two co...,"[ABC TV, DOCUMENTARY, LIFESTYLE, SCIENCE]",https://cdn.iview.abc.net.au/thumbs/i/do/DO202...,[Host Craig Reucassel],"[abc1, docs, lifestyle, factual, nature, weath...",2021-03-28 18:28:08,PG,1500.0,NaN,,Ready Together
4,1,7.0,367,Documentary,Capturing Cricket: Steve Waugh In India,NaN,Capturing Cricket: Steve Waugh In India,"Steve Waugh, the mastermind behind Australia's...","Steve Waugh, the mastermind behind Australia's...","[ABC TV, DOCUMENTARY, SPORT]",https://cdn.iview.abc.net.au/thumbs/i/rf/RF192...,[Host Steve Waugh],"[abc1, australia, docs, factual, sport, divers...",2020-11-17 21:30:47,PG,3407.0,NaN,,Capturing Cricket: Steve Waugh In India
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,5,0.0,232,Documentary,Can We Save The Reef?,NaN,Can We Save The Reef?,An epic story of Australian and international ...,An epic story of Australian and international ...,"[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/sc/SC160...,[],"[abc1, docs, nature, science, science-week, cl...",2018-05-12 14:58:55,G,3428.0,NaN,,Can We Save The Reef?
142,5,0.0,376,Documentary,Brazen Hussies,NaN,Brazen Hussies,A revealing documentary celebrating the legacy...,A revealing documentary celebrating the legacy...,"[ABC TV, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/do/DO190...,[Director Catherine Dwyer],"[abc1, featured, docs, australia, factual, his...",2022-03-08 01:04:06,M,5318.0,NaN,,Brazen Hussies
143,5,0.0,674,Documentary,Fighting Spirit: Wheeling Diggers' Invictus Ga...,NaN,Fighting Spirit: Wheeling Diggers' Invictus Ga...,This deeply moving documentary goes behind-the...,This deeply moving documentary goes behind-the...,"[ABC TV, SPORT, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/do/DO171...,[],"[abc1, aussie, sport, docs, factual, inspirati...",2018-10-16 21:28:00,M,3442.0,NaN,,Fighting Spirit: Wheeling Diggers' Invictus Ga...
144,5,0.0,1005,Documentary,The Art of Remembrance,NaN,The Art of Remembrance,Nate Byrne explores the role of paintings and ...,Nate Byrne explores the role of paintings and ...,"[ABC TV, DOCUMENTARY, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/rf/RF200...,[Host Nate Byrne],"[abc1, docs, factual, australia, history, hist...",2021-04-25 18:28:14,PG,1594.0,NaN,,The Art of Remembrance


In [26]:
#shows
recommended_shows= pd.merge(top3_show, expat_library_shows, on=['cluster'], how='left')
recommended_shows

,user_id,cluster,title,id,category,description,description2,directors_actors,publication_date,rating,duration_sec,episodes_ids,tags,tags2
0,1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2.0,Australia Remastered: Wild Australians,11589.0,Family,Explore some of Australia's most iconic and fa...,"[Echidnas and platypuses are unique, the only ...",[Host Aaron Pedersen],2020-08-30 18:57:00,G,"[3310.0, 3330.0, 3285.0, 3185.0, 3230.0, 3294.0]","[1990, 2401, 2805, 2863, 3397, 3614]","[ABC TV, ABC ME, DOCUMENTARY, FAMILY]","[a:retirees, docs, wildlife, science-week, ns:..."
2,1,2.0,Australia's Prime Ministers,11593.0,Education,"This series profiles Australia's leaders, feat...","[This series profiles Australia's leaders, fea...",[],2022-02-28 09:00:00,G,"[183.0, 119.0, 186.0, 180.0, 199.0, 227.0, 157...","[12656, 12957, 12961, 13087, 13091, 13161, 133...","[ABC ME, EDUCATION, DOCUMENTARY]","[secondary-history, gough-whitlam, docs, prima..."
3,1,2.0,Employable Me Australia,11705.0,Documentary,The Rose d'Or nominated series follows six ind...,"[This uplifting, warm, and insightful series s...",[],2019-04-09 21:32:30,PG,"[239.0, 168.0, 3566.0, 3579.0, 235.0, 233.0, 3...","[9463, 9466, 9566, 9618, 9667, 9668, 9727, 974...","[ABC TV, DOCUMENTARY]","[docs, real-life, disability, endearing, heart..."
4,1,4.0,Australia Debates,11585.0,Panel Discussion,The funniest minds in Australia debate the fin...,"[Tom Cashman, Demi Lardner and Nat Damena argu...",[Host Nikki Britton],2021-06-28 21:33:30,M,"[2579.0, 2605.0, 2492.0]","[26, 49, 225]","[ABC TV Plus, PANEL & DISCUSSION]","[a:established-parents, abc2, ns:be-informed, ..."
5,1,4.0,Australia Remastered: Forces of Nature,11587.0,Family,"Hosted and narrated by Aaron Pedersen, this se...",[Cyclones burst upon Australia's tropical coas...,[Host Aaron Pedersen],2022-01-30 18:04:10,G,"[3230.0, 3260.0, 3195.0, 3240.0]","[1861, 2547, 2688, 3738]","[ABC TV, DOCUMENTARY, REGIONAL AUSTRALIA, FAMILY]","[docs, environment, regional, natural-disaster..."
6,1,4.0,Deep Dive Into Australia's Ocean Odyssey,11686.0,Education,"Focussing on marine life, interconnectedness o...",[Emily Jateff from Australian National Maritim...,[],2020-08-17 11:56:21,PG,"[1206.0, 1090.0, 1271.0]","[13951, 14305, 14452]","[ABC ME, EDUCATION]","[abc3, secondary-maths, education]"
7,1,4.0,Gardening Australia - My Garden Path,11741.0,Arts,Gardening Australia is hosted by Costa Georgia...,[We meet a garden designer who celebrates text...,[],2021-02-12 07:00:00,G,"[366.0, 372.0, 407.0, 509.0, 385.0, 467.0, 401...","[11753, 11758, 11776, 11787, 11794, 11820, 118...","[ABC TV, LIFESTYLE, REGIONAL AUSTRALIA]","[western-australia, victoria, darwin, regional..."
8,1,4.0,Griff's Great Australian Rail Trip,11757.0,Documentary,British comedian and travel enthusiast Griff R...,[Journeying from Brisbane to Cairns aboard the...,[Host Griff Rhys Jones],2021-06-30 07:00:00,G,"[2829.0, 2785.0, 2831.0, 2768.0, 2821.0, 2832.0]","[9483, 9707, 9899, 10140, 10226, 10551]","[ABC TV, LIFESTYLE, DOCUMENTARY]","[docs, travel, australia, uk, lifestyle, abc1,..."
9,1,4.0,Prepping Australia,11966.0,Documentary,Meet real Australian Preppers bunkering down f...,[After the 2020 supermarket shortages left her...,[],2021-11-16 22:30:00,G,"[601.0, 909.0, 934.0]","[9481, 9684, 9738]","[ABC TV Plus, DOCUMENTARY]","[abc2, conspiracy, offbeat, docs, environment,..."


In [27]:
#drop intersections of recommended and activity by user_id and content_id
recommended = recommended[~recommended[['user_id', 'id']].apply(frozenset, axis=1).isin(activity[['user_id', 'content_id']].apply(frozenset, axis=1))]
recommended

,user_id,cluster,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title
0,1,7.0,595,Family,Freeman,NaN,Freeman,The story of a nation coming together around I...,The story of a nation coming together around I...,"[ABC TV, ABC ME, INDIGENOUS, DOCUMENTARY, SPOR...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF190...,[Director Laurence Billiet],"[abc1, australia, indigenous, docs, factual, f...",2021-06-30 07:05:00,G,3480.0,NaN,,Freeman
1,1,7.0,1979,Family,Sea Lions: Life By A Whisker,NaN,Sea Lions: Life By A Whisker,Take an intimate journey inside the colony of ...,Take an intimate journey inside the colony of ...,"[ABC ME, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW284...,[Narrator Sam Neill],"[abc3, family-viewing-abcme, 3featured, family...",2022-03-16 09:45:00,G,2439.0,NaN,,Sea Lions: Life By A Whisker
2,1,7.0,32,Documentary,Fly Me To The Moon,NaN,Fly Me To The Moon,The Moon Landing was the iconic moment of our ...,The Moon Landing was the iconic moment of our ...,"[ABC NEWS, DOCUMENTARY, SCIENCE]",https://cdn.iview.abc.net.au/thumbs/i/nc/NC193...,[],"[news24, news, space, docs, science, featured]",2021-04-07 13:00:00,PG,3480.0,NaN,,Fly Me To The Moon
3,1,7.0,71,Documentary,Ready Together,NaN,Ready Together,Craig Reucassel presents the stories of two co...,Craig Reucassel presents the stories of two co...,"[ABC TV, DOCUMENTARY, LIFESTYLE, SCIENCE]",https://cdn.iview.abc.net.au/thumbs/i/do/DO202...,[Host Craig Reucassel],"[abc1, docs, lifestyle, factual, nature, weath...",2021-03-28 18:28:08,PG,1500.0,NaN,,Ready Together
4,1,7.0,367,Documentary,Capturing Cricket: Steve Waugh In India,NaN,Capturing Cricket: Steve Waugh In India,"Steve Waugh, the mastermind behind Australia's...","Steve Waugh, the mastermind behind Australia's...","[ABC TV, DOCUMENTARY, SPORT]",https://cdn.iview.abc.net.au/thumbs/i/rf/RF192...,[Host Steve Waugh],"[abc1, australia, docs, factual, sport, divers...",2020-11-17 21:30:47,PG,3407.0,NaN,,Capturing Cricket: Steve Waugh In India
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,5,0.0,44,Documentary,Weird Australia,NaN,Weird Australia,"With the help of eyewitness accounts, qualifie...","With the help of eyewitness accounts, qualifie...","[ABC TV, SCIENCE, REGIONAL AUSTRALIA, DOCUMENT...",https://cdn.iview.abc.net.au/thumbs/i/sq/SQ210...,[Host Moataz Hamde],"[abc1, science, australia, nature, wildlife, e...",2022-01-01 09:29:00,PG,1802.0,NaN,,Weird Australia
142,5,0.0,376,Documentary,Brazen Hussies,NaN,Brazen Hussies,A revealing documentary celebrating the legacy...,A revealing documentary celebrating the legacy...,"[ABC TV, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/do/DO190...,[Director Catherine Dwyer],"[abc1, featured, docs, australia, factual, his...",2022-03-08 01:04:06,M,5318.0,NaN,,Brazen Hussies
143,5,0.0,674,Documentary,Fighting Spirit: Wheeling Diggers' Invictus Ga...,NaN,Fighting Spirit: Wheeling Diggers' Invictus Ga...,This deeply moving documentary goes behind-the...,This deeply moving documentary goes behind-the...,"[ABC TV, SPORT, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/do/DO171...,[],"[abc1, aussie, sport, docs, factual, inspirati...",2018-10-16 21:28:00,M,3442.0,NaN,,Fighting Spirit: Wheeling Diggers' Invictus Ga...
144,5,0.0,1005,Documentary,The Art of Remembrance,NaN,The Art of Remembrance,Nate Byrne explores the role of paintings and ...,Nate Byrne explores the role of paintings and ...,"[ABC TV, DOCUMENTARY, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/rf/RF200...,[Host Nate Byrne],"[abc1, docs, factual, australia, history, hist...",2021-04-25 18:28:14,PG,1594.0,NaN,,The Art of Remembrance


In [28]:
recommended_shows = recommended_shows[~recommended_shows[['user_id', 'id']].apply(frozenset, axis=1).isin(activity[['user_id', 'content_id']].apply(frozenset, axis=1))]
recommended_shows

,user_id,cluster,title,id,category,description,description2,directors_actors,publication_date,rating,duration_sec,episodes_ids,tags,tags2
0,1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2.0,Australia's Prime Ministers,11593.0,Education,"This series profiles Australia's leaders, feat...","[This series profiles Australia's leaders, fea...",[],2022-02-28 09:00:00,G,"[183.0, 119.0, 186.0, 180.0, 199.0, 227.0, 157...","[12656, 12957, 12961, 13087, 13091, 13161, 133...","[ABC ME, EDUCATION, DOCUMENTARY]","[secondary-history, gough-whitlam, docs, prima..."
3,1,2.0,Employable Me Australia,11705.0,Documentary,The Rose d'Or nominated series follows six ind...,"[This uplifting, warm, and insightful series s...",[],2019-04-09 21:32:30,PG,"[239.0, 168.0, 3566.0, 3579.0, 235.0, 233.0, 3...","[9463, 9466, 9566, 9618, 9667, 9668, 9727, 974...","[ABC TV, DOCUMENTARY]","[docs, real-life, disability, endearing, heart..."
4,1,4.0,Australia Debates,11585.0,Panel Discussion,The funniest minds in Australia debate the fin...,"[Tom Cashman, Demi Lardner and Nat Damena argu...",[Host Nikki Britton],2021-06-28 21:33:30,M,"[2579.0, 2605.0, 2492.0]","[26, 49, 225]","[ABC TV Plus, PANEL & DISCUSSION]","[a:established-parents, abc2, ns:be-informed, ..."
5,1,4.0,Australia Remastered: Forces of Nature,11587.0,Family,"Hosted and narrated by Aaron Pedersen, this se...",[Cyclones burst upon Australia's tropical coas...,[Host Aaron Pedersen],2022-01-30 18:04:10,G,"[3230.0, 3260.0, 3195.0, 3240.0]","[1861, 2547, 2688, 3738]","[ABC TV, DOCUMENTARY, REGIONAL AUSTRALIA, FAMILY]","[docs, environment, regional, natural-disaster..."
6,1,4.0,Deep Dive Into Australia's Ocean Odyssey,11686.0,Education,"Focussing on marine life, interconnectedness o...",[Emily Jateff from Australian National Maritim...,[],2020-08-17 11:56:21,PG,"[1206.0, 1090.0, 1271.0]","[13951, 14305, 14452]","[ABC ME, EDUCATION]","[abc3, secondary-maths, education]"
9,1,4.0,Prepping Australia,11966.0,Documentary,Meet real Australian Preppers bunkering down f...,[After the 2020 supermarket shortages left her...,[],2021-11-16 22:30:00,G,"[601.0, 909.0, 934.0]","[9481, 9684, 9738]","[ABC TV Plus, DOCUMENTARY]","[abc2, conspiracy, offbeat, docs, environment,..."
10,1,4.0,The Art Of Australia,12055.0,Documentary,The Art of Australia explores how art and arti...,[The story of how art helped European settlers...,[Host Edmund Capon],2021-06-18 07:00:00,G,"[3390.0, 3423.0, 3402.0]","[9720, 10142, 10214]","[ABC TV, ABC ARTS, ARTS & CULTURE, DOCUMENTARY]","[a:retirees, revealing, indigenous-history, do..."
11,2,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,2,1.0,Australia Remastered: Wild Wars,11591.0,Family,Life in Australia is a battle. Shaped by the c...,[While forests only cover 16% of Australia's l...,[Host Aaron Pedersen],2021-03-02 22:33:27,G,"[3235.0, 3265.0, 3280.0, 3240.0]","[2364, 3015, 3501, 3640]","[ABC TV, DOCUMENTARY, FAMILY]","[a:retirees, docs, wildlife, science-week, ns:..."


In [29]:
#choose 2 rows per each user_id and each cluster
recommended_films = recommended.groupby(['user_id', 'cluster']).head(2)
recommended_films

,user_id,cluster,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title
0,1,7.0,595,Family,Freeman,NaN,Freeman,The story of a nation coming together around I...,The story of a nation coming together around I...,"[ABC TV, ABC ME, INDIGENOUS, DOCUMENTARY, SPOR...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF190...,[Director Laurence Billiet],"[abc1, australia, indigenous, docs, factual, f...",2021-06-30 07:05:00,G,3480.0,NaN,,Freeman
1,1,7.0,1979,Family,Sea Lions: Life By A Whisker,NaN,Sea Lions: Life By A Whisker,Take an intimate journey inside the colony of ...,Take an intimate journey inside the colony of ...,"[ABC ME, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW284...,[Narrator Sam Neill],"[abc3, family-viewing-abcme, 3featured, family...",2022-03-16 09:45:00,G,2439.0,NaN,,Sea Lions: Life By A Whisker
13,1,0.0,404,Family,Can We Save The Reef?,NaN,Can We Save The Reef?,An epic story of Australian and international ...,An epic story of Australian and international ...,"[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/sc/SC160...,[],"[abc1, docs, nature, science, science-week, cl...",2018-05-12 14:58:55,G,3428.0,NaN,,Can We Save The Reef?
14,1,0.0,44,Documentary,Weird Australia,NaN,Weird Australia,"With the help of eyewitness accounts, qualifie...","With the help of eyewitness accounts, qualifie...","[ABC TV, SCIENCE, REGIONAL AUSTRALIA, DOCUMENT...",https://cdn.iview.abc.net.au/thumbs/i/sq/SQ210...,[Host Moataz Hamde],"[abc1, science, australia, nature, wildlife, e...",2022-01-01 09:29:00,PG,1802.0,NaN,,Weird Australia
21,1,4.0,1073,Family,Sleeping Beauty,NaN,Sleeping Beauty,The Australian Ballet presents Sleeping Beauty...,The Australian Ballet presents Sleeping Beauty...,"[ABC TV Plus, ARTS & CULTURE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW229...,[],"[abc2, arts, entertainment, concert, ballet, c...",2021-06-18 07:00:00,G,7656.0,NaN,,Sleeping Beauty
23,1,4.0,1385,Family,Coppelia,NaN,Coppelia,"A sparkling tale of magic and mischief, Coppel...","A sparkling tale of magic and mischief, Coppel...","[ABC TV Plus, ABC ARTS, ARTS & CULTURE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW230...,[],"[abc2, arts, abcarts, ballet, opera, entertain...",2021-06-18 07:00:00,G,6511.0,NaN,,Coppelia
41,2,12.0,938,Documentary,Dubboo: Life Of A Songman,NaN,Dubboo: Life Of A Songman,Bangarra Dance Theatre joins forces with some ...,Bangarra Dance Theatre joins forces with some ...,"[ABC TV Plus, ABC ARTS, INDIGENOUS, ARTS & CUL...",https://cdn.iview.abc.net.au/thumbs/i/ac/AC194...,"[Cast Archie Roach, Ursula Yovich, Djakapurra ...","[abc2, indigenous, abcarts, arts, docs, factua...",2021-07-07 22:00:55,G,3379.0,NaN,,Dubboo: Life Of A Songman
42,2,12.0,619,Drama,Cliffy,NaN,Cliffy,The remarkable story of Cliffy; an Australian ...,The remarkable story of Cliffy an Australian f...,"[ABC TV, ABC TV Plus, MOVIES, DRAMA]",https://cdn.iview.abc.net.au/thumbs/i/dr/DR110...,[Director Dean Murphy],"[abc1, abc2, film, drama, comedy-drama, inspir...",2021-06-11 07:00:00,PG,5509.0,NaN,,Cliffy
45,2,2.0,424,Family,Stackorama!,NaN,Stackorama!,"Jaydyn Coggins, the best cup stacker in Austra...","Jaydyn Coggins, the best cup stacker in Austra...","[ABC TV Plus, DOCUMENTARY, SPORT, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/do/DO191...,[],"[abc2, docs, factual, sport, endearing, disabi...",2021-03-31 21:45:07,PG,3586.0,NaN,,Stackorama!
46,2,2.0,242,Documentary,Stackorama!,NaN,Stackorama!,"Jaydyn Coggins, the best cup stacker in Austra...","Jaydyn Coggins, the best cup stacker in Austra...","[ABC TV Plus, DOCUMENTARY, SPORT, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/do/DO191...,[],"[abc2, docs, factual, sport, endearing, disabi...",2021-03-31 21:45:07,PG,3586.0,NaN,,Stackorama!


In [30]:
recommended_shows = recommended_shows.groupby(['user_id', 'cluster']).head(2)
recommended_shows

,user_id,cluster,title,id,category,description,description2,directors_actors,publication_date,rating,duration_sec,episodes_ids,tags,tags2
0,1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,2.0,Australia's Prime Ministers,11593.0,Education,"This series profiles Australia's leaders, feat...","[This series profiles Australia's leaders, fea...",[],2022-02-28 09:00:00,G,"[183.0, 119.0, 186.0, 180.0, 199.0, 227.0, 157...","[12656, 12957, 12961, 13087, 13091, 13161, 133...","[ABC ME, EDUCATION, DOCUMENTARY]","[secondary-history, gough-whitlam, docs, prima..."
3,1,2.0,Employable Me Australia,11705.0,Documentary,The Rose d'Or nominated series follows six ind...,"[This uplifting, warm, and insightful series s...",[],2019-04-09 21:32:30,PG,"[239.0, 168.0, 3566.0, 3579.0, 235.0, 233.0, 3...","[9463, 9466, 9566, 9618, 9667, 9668, 9727, 974...","[ABC TV, DOCUMENTARY]","[docs, real-life, disability, endearing, heart..."
4,1,4.0,Australia Debates,11585.0,Panel Discussion,The funniest minds in Australia debate the fin...,"[Tom Cashman, Demi Lardner and Nat Damena argu...",[Host Nikki Britton],2021-06-28 21:33:30,M,"[2579.0, 2605.0, 2492.0]","[26, 49, 225]","[ABC TV Plus, PANEL & DISCUSSION]","[a:established-parents, abc2, ns:be-informed, ..."
5,1,4.0,Australia Remastered: Forces of Nature,11587.0,Family,"Hosted and narrated by Aaron Pedersen, this se...",[Cyclones burst upon Australia's tropical coas...,[Host Aaron Pedersen],2022-01-30 18:04:10,G,"[3230.0, 3260.0, 3195.0, 3240.0]","[1861, 2547, 2688, 3738]","[ABC TV, DOCUMENTARY, REGIONAL AUSTRALIA, FAMILY]","[docs, environment, regional, natural-disaster..."
11,2,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,2,1.0,Australia Remastered: Wild Wars,11591.0,Family,Life in Australia is a battle. Shaped by the c...,[While forests only cover 16% of Australia's l...,[Host Aaron Pedersen],2021-03-02 22:33:27,G,"[3235.0, 3265.0, 3280.0, 3240.0]","[2364, 3015, 3501, 3640]","[ABC TV, DOCUMENTARY, FAMILY]","[a:retirees, docs, wildlife, science-week, ns:..."
13,2,4.0,Australia Debates,11585.0,Panel Discussion,The funniest minds in Australia debate the fin...,"[Tom Cashman, Demi Lardner and Nat Damena argu...",[Host Nikki Britton],2021-06-28 21:33:30,M,"[2579.0, 2605.0, 2492.0]","[26, 49, 225]","[ABC TV Plus, PANEL & DISCUSSION]","[a:established-parents, abc2, ns:be-informed, ..."
14,2,4.0,Australia Remastered: Forces of Nature,11587.0,Family,"Hosted and narrated by Aaron Pedersen, this se...",[Cyclones burst upon Australia's tropical coas...,[Host Aaron Pedersen],2022-01-30 18:04:10,G,"[3230.0, 3260.0, 3195.0, 3240.0]","[1861, 2547, 2688, 3738]","[ABC TV, DOCUMENTARY, REGIONAL AUSTRALIA, FAMILY]","[docs, environment, regional, natural-disaster..."
20,3,4.0,Australia Debates,11585.0,Panel Discussion,The funniest minds in Australia debate the fin...,"[Tom Cashman, Demi Lardner and Nat Damena argu...",[Host Nikki Britton],2021-06-28 21:33:30,M,"[2579.0, 2605.0, 2492.0]","[26, 49, 225]","[ABC TV Plus, PANEL & DISCUSSION]","[a:established-parents, abc2, ns:be-informed, ..."
